# Introduction to Python 3: Classes
## Luca de Alfaro
Copyright Luca de Alfaro, 2018-21.  CC-BY-NC License.



Prepared on: Fri Jul 30 17:08:49 2021

This is a book chapter; it is not a homework assignment.  
Do not submit it as a solution to a homework assignment; you would receive no credit.


## Classes

Here is a simple example of a class declaration.


In [1]:
class Product(object):

    def __init__(self, name, price=0., quantity=0):
        """In the initializer, you should define the values that each object
        has.  Here, 'self' means, the object."""
        self.name = name
        self.price = price
        self.quantity = quantity

    def __repr__(self):
        """Represents a class element in a reasonable way.
        Note the format statement below to help produce a string."""
        return "Hello, I am a {} and cost ${}; you have {} of me".format(
            self.name, self.price, self.quantity
        )


    def inflation(self, x):
        """Increases the price by a factor x.
        Note how self is always the first argument of methods; otherwise,
        you would not know to which object to apply the operations."""
        self.price *= x

    def value(self):
        """Total value of products of this type."""
        return self.price * self.quantity



Let's make a list of products.


In [2]:
cart = [
    Product('Pear', price=1.99, quantity=10),
    Product('Apple', price=0.99, quantity=15),
    Product('Onion', price=1.49, quantity=57)
]


We can print it; the representation is given by `__repr__`. 


In [3]:
for p in cart:
    print(p)


Hello, I am a Pear and cost $1.99; you have 10 of me
Hello, I am a Apple and cost $0.99; you have 15 of me
Hello, I am a Onion and cost $1.49; you have 57 of me


What if you buy more apples?  

The proper way would be to define a buy method, and write 
something like `p.buy(10)` to buy 10 more.  But in Python, there is 
nothing to prevent you from accessing object variables directly.



In [4]:
def double_the_cart(c):
    for p in c:
        p.quantity *= 2

double_the_cart(cart)

def print_cart(c):
    for p in c:
        print(p)

print_cart(cart)


Hello, I am a Pear and cost $1.99; you have 20 of me
Hello, I am a Apple and cost $0.99; you have 30 of me
Hello, I am a Onion and cost $1.49; you have 114 of me


## A simple event-based simulator

Let's try to put everything together and design a simple event-based simulator. 

Every event will have a time at which it happens.  When it happens, it will generate two things: a string that is printed, and a list (possibly empty) of subsequent events. 

Let us write the code for three event types: one that occurs only once, one that occurs periodically with a certain delay between occurrences forever, and one that occurs periodically, but has a specified maximum number of occurrences.

Before we do it, we have to remedy one of the few silly 
choices in the design of Python.
There is apparently no sign function! 
This is so silly that I can't resist defining one. 


In [5]:
def sign(x):
    if x < 0:
        return -1
    elif x > 0:
        return 1
    else:
        return 0


In [6]:
class GenericEvent(object):

    def __init__(self, name, time):
        self.name = name
        self.time = time

    def __repr__(self):
        return "Event {} of type {} will occurr at {}".format(
            self.name,
            type(self),
            self.time.isoformat()
        )

    def __lt__(self, other):
        """To sort events according to their time, we need to
        implement the __lt__ operator.  This will be used by heapq later."""
        return self.time < other.time

    def _effect(self):
        """In Python, methods that are supposed to be accessed only within
        the class are prepended with _.  Note that this is just a convention;
        nothing prevents you from calling these methods from outside the class.
        """
        print("At {}: {}".format(self.time.isoformat(), self.name))

    def do(self):
        """You are supposed to define what happens in each subclass."""
        raise NotImplementedError

# We need the datetime module to process times.
import datetime
e = GenericEvent('Sun shines', datetime.datetime.now() + datetime.timedelta(hours=1))
e


Event Sun shines of type <class '__main__.GenericEvent'> will occurr at 2021-07-31T00:20:07.180081

Let's define an event that happens once only.


In [7]:
class OnceOnlyEvent(GenericEvent):
    """OnceOnlyEvent extends GenericEvent, and so it inherits all of its
    methods, including __repr__, __comp__, _effect."""

    def __init__(self, name, time):
        # We simply define an element of the superclass.
        # Here, super(OnceOnlyEvent, self) is the Python way for getting
        # access to the superclass methods from a subclass.
        super().__init__(name, time)

    def do(self):
        self._effect()
        # No other events are generated.
        return []


We can define periodic events, which occur forever with a given period: 

In [8]:
class InfinitePeriodicEvent(GenericEvent):
    """This is a periodic event."""

    def __init__(self, name, time, periodicity):
        """time is a datetime object; periodicity is expressed as a timedelta object."""
        super().__init__(name, time)
        self.periodicity = periodicity

    def do(self):
        self._effect()
        # Generates and returns the next occurrence of the event.
        next_event = PeriodicEvent(
            self.name,
            self.time + self.periodicity,
            self.periodicity
        )
        return [next_event]




We can also define periodic events with a finite number of occurrences: you may be familiar with them from scheduling with calendars: 

In [9]:
class PeriodicEvent(GenericEvent):
    """This is a periodic event like above, except that it has
    an optional maximum number of occurrences."""

    def __init__(self, name, time, periodicity, num_occurrences=None):
        """
        Let's document this constructor a bit better.
        @param name: name of the event.
        @param time: time of first occurrence of the event (datetime object).
        @param periodicity: periodicity of the event (timedelta object).
        @param num_occurrences: number of future occurrences of the event.
            If None, then infinite future occurrences can happen.

        """
        assert num_occurrences is None or num_occurrences > 0
        # We don't want to go back in time!
        assert periodicity.total_seconds() > 0.
        super().__init__(name, time)
        self.periodicity = periodicity
        self.num_occurrences = num_occurrences

    def do(self):
        self._effect()
        # Generates and returns the next occurrence of the event.
        if self.num_occurrences is None or self.num_occurrences > 1:
            return [PeriodicEvent(
                self.name,
                self.time + self.periodicity,
                self.periodicity,
                num_occurrences = None if self.num_occurrences is None
                                  else self.num_occurrences - 1
            )]
        else:
            return []


Great.  Now, let's define our discrete event simulator.  It will be a class, have a method to add new events to it, and it will have a method step(), which causes the next event to occur. 

In order to quickly determine which one is the next event, 
we will store events in a priority queue.  This is implemented via the heapq module, whose documentation is at https://docs.python.org/2/library/heapq.html

In [10]:
class EventSimulator(object):

    def __init__(self, event_list=None):
        # It would be better to use a heap, but for simplicity,
        # we use a list.
        self.event_list = event_list or []
        # .sort() uses __lt__ for GenericEvent;
        # see https://docs.python.org/3/howto/sorting.html#odd-and-ends
        self.event_list.sort()
        # Or I could also sort wrt a key.
        # self.event_list.sort(key = lambda event : event.time)

    def add_event(self, e):
        """Adds an event e, maintaining the heap invariant."""
        self.event_list.append(e)
        # .sort() uses __lt__ for GenericEvent;
        # see https://docs.python.org/3/howto/sorting.html#odd-and-ends
        self.event_list.sort()

    def step(self):
        """Performs one step of the event simulator."""
        # Gets the first event.
        e = self.event_list.pop(0)
        # Causes e to happen.
        generated_events = e.do()
        # And inserts the resulting events into the heap of future events.
        for ge in generated_events:
            self.add_event(ge)


That's all there is to it.  Now let's try how it works.
We generate a couple of events that happen only once: 


In [11]:
now = datetime.datetime.now()
ten_secs = datetime.timedelta(seconds=10)
twentyfive_secs = datetime.timedelta(seconds=25)

once1 = OnceOnlyEvent("once1", now + ten_secs)
once2 = OnceOnlyEvent("once2", now + twentyfive_secs)


Let's also define two periodic events, one with 3 occurrences,
the other with infinite occurrences.


In [12]:
two_secs = datetime.timedelta(seconds=2)
three_secs = datetime.timedelta(seconds=3)

periodic1 = PeriodicEvent("periodic1", now + ten_secs, three_secs, num_occurrences=7)
periodic2 = PeriodicEvent("periodic2", now + twentyfive_secs, three_secs)


Let's create our event simulator. 


In [13]:
sim = EventSimulator([once1, once2, periodic1, periodic2])


In [14]:
sim.step()


At 2021-07-30T23:22:20.336884: once1


What's in the event queue? 


In [15]:
sim.event_list


[Event periodic1 of type <class '__main__.PeriodicEvent'> will occurr at 2021-07-30T23:22:20.336884,
 Event once2 of type <class '__main__.OnceOnlyEvent'> will occurr at 2021-07-30T23:22:35.336884,
 Event periodic2 of type <class '__main__.PeriodicEvent'> will occurr at 2021-07-30T23:22:35.336884]

Let's do 20 steps now.


In [16]:
for _ in range(20):
    sim.step()
sim.event_list


At 2021-07-30T23:22:20.336884: periodic1
At 2021-07-30T23:22:23.336884: periodic1
At 2021-07-30T23:22:26.336884: periodic1
At 2021-07-30T23:22:29.336884: periodic1
At 2021-07-30T23:22:32.336884: periodic1
At 2021-07-30T23:22:35.336884: once2
At 2021-07-30T23:22:35.336884: periodic2
At 2021-07-30T23:22:35.336884: periodic1
At 2021-07-30T23:22:38.336884: periodic2
At 2021-07-30T23:22:38.336884: periodic1
At 2021-07-30T23:22:41.336884: periodic2
At 2021-07-30T23:22:44.336884: periodic2
At 2021-07-30T23:22:47.336884: periodic2
At 2021-07-30T23:22:50.336884: periodic2
At 2021-07-30T23:22:53.336884: periodic2
At 2021-07-30T23:22:56.336884: periodic2
At 2021-07-30T23:22:59.336884: periodic2
At 2021-07-30T23:23:02.336884: periodic2
At 2021-07-30T23:23:05.336884: periodic2
At 2021-07-30T23:23:08.336884: periodic2


[Event periodic2 of type <class '__main__.PeriodicEvent'> will occurr at 2021-07-30T23:23:11.336884]

*That's All, Folks!*